# Oak-D Aruco Tag Distance Test

Use this notebook to test how far away the Oak-d camera can detect an Aruco tag. We've found that a high definition camera (1920x1080) can detect a tag from about 35 feet away, but Oak-d camera has 4k resolution, so hopefully it can sense the tags from a greater distance (the target is 70 feet).

## Setup

You'll need to set up the DepthAI program (see Adrien's setup steps [here](https://github.com/SJSURoboticsTeam/urc-intelligent_systems-2023/blob/main/Vision/README.md)). You will also need to install OpenCV-Contrib version 4.6.0.66 (a lower version is okay, but a higher has breaking API changes).

In [1]:
#!pip install opencv-contrib-python==4.6.0.66

In [2]:
import cv2
import numpy as np
from collections import deque
import os
import glob
import depthai
import sys
sys.path.append('../../')
from modules.OAKD import OakD

In [3]:
# load the 4x4_50 aruco tag dictionary
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

## Camera



In [4]:
cap = OakD(depthai.ColorCameraProperties.SensorResolution.THE_720_P)

mtx = cap.mtx
dist = cap.dist

[194430102186771300] [1.3] [1.141] [ColorCamera(0)] [warning] Unsupported resolution set for detected camera IMX378/214, needs 1080_P / 4_K / 12_MP. Defaulting to 1080_P


## Detecting Aruco Tags

This is the main loop where we try to find aruco tags. It'll draw a green bounding box around any tag it detects. It also looks through the last couple frames to see if it can detect a tag in any of those frames. This helps its accuracy at long distances because the odds of it detecting a tag in single frame isn't great, but over many frames, it's pretty decent. Press 'q' to stop the loop.

In [5]:
frame_queue = deque(maxlen=1)

while True:
    _, frame = cap.read()

    frame_queue.append(frame)

    corners, ids = [], []

    for test_frame in frame_queue:
        # detect the aruco markers in the image
        possible_corners, possible_ids, rejectedImgPoints = cv2.aruco.detectMarkers(test_frame, arucoDict,parameters=parameters)
        if len(possible_corners) > 0:
            corners.extend(possible_corners)
            ids.extend(possible_ids.flatten())
            break

    for corner, tag_id in zip(corners, ids):
        # loop over the detected ArUCo corners
        #for (markerCorner, markerID) in zip(corners, ids):
        # extract the marker corners (which are always returned in
        # top-left, top-right, bottom-right, and bottom-left order)
        # corners = markerCorner.reshape((4, 2))

        (topLeft, topRight, bottomRight, bottomLeft) = corner[0]
        # convert each of the (x, y)-coordinate pairs to integers
        topRight = (int(topRight[0]), int(topRight[1]))
        bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
        bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
        topLeft = (int(topLeft[0]), int(topLeft[1]))
        # draw the bounding box of the ArUCo detection
        cv2.line(frame, topLeft, topRight, (0, 255, 0), 2)
        cv2.line(frame, topRight, bottomRight, (0, 255, 0), 2)
        cv2.line(frame, bottomRight, bottomLeft, (0, 255, 0), 2)
        cv2.line(frame, bottomLeft, topLeft, (0, 255, 0), 2)
        # compute and draw the center (x, y)-coordinates of the ArUco
        # marker
        cX = int((topLeft[0] + bottomRight[0]) / 2.0)
        cY = int((topLeft[1] + bottomRight[1]) / 2.0)
        cv2.circle(frame, (cX, cY), 4, (0, 0, 255), -1)
        # draw the ArUco marker ID on the frame
        cv2.putText(frame, str(tag_id),
            (topLeft[0], topLeft[1] - 15), cv2.FONT_HERSHEY_SIMPLEX,
            0.5, (0, 255, 0), 2)

        # estimate the distance of the marker from the camera
        markerSizeInM = .20

        rvect, tvect, _ = cv2.aruco.estimatePoseSingleMarkers(corner, markerSizeInM, mtx, dist)

        distance = np.linalg.norm(tvect[0][0])
        # show the distance in the frame (limit the distance to 2 decimal places))
        cv2.putText(frame, str(distance), (topLeft[0], topLeft[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    if frame is not None:

        #frame = cv2.resize(frame, None, fx=.25, fy=.25)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        #if the `q` key was pressed, break from the loop
        if key == ord("q"):
           break

In [ ]:
# close all windows
cv2.destroyAllWindows()